In [644]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression ,LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error ,accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from statistics import mode
import warnings
warnings.filterwarnings("ignore")
gnd = pd.read_csv("/Users/hajiaga/Downloads/titanic/gender_submission.csv")
tr = pd.read_csv("/Users/hajiaga/Downloads/titanic/train.csv")
te = pd.read_csv("/Users/hajiaga/Downloads/titanic/test.csv")

In [645]:
data = pd.concat([tr,te],axis = 0)
data.drop(["PassengerId","Name","Ticket","Cabin","Embarked"],axis=1,inplace=True)


In [646]:
def detect_outliers(data,col):
    Q1 = data[col].quantile(0.05)
    Q2 = data[col].quantile(0.95)
    IQR = Q2-Q1
    lower_bound = Q1-IQR*1.5
    higher_bound = Q2+IQR*1.5
    data.drop(data[data[col]>higher_bound].index ,axis =0, inplace=True)
    data.drop(data[data[col]<lower_bound].index , axis = 0, inplace=True)


In [647]:

# I used this code to look at my outlier detecter's impoct on my data ,but then deleted because it will be a big output
a = np.nanmedian(data["Age"])
data["Age"] = data["Age"].fillna(a)
data["Fare"].fillna(np.nanmedian(data["Fare"]),inplace=True)

In [648]:
def evaluate(model,X,Y):
    X_tr,X_te,y_tr,y_te = train_test_split(X,Y,test_size=0.8,random_state=42,shuffle=True)
    model.fit(X_tr,y_te)
    pr = model.predict(X_te)
    rslt = accuracy_score(y_te,pr)
    return rslt

In [649]:
le = LabelEncoder()
data["Sex"] = le.fit_transform(data["Sex"])

In [650]:
num_cols = data.select_dtypes("number").columns
train = data[data["Survived"].isna() == False]
test = data[data["Survived"].isna()]
for i in num_cols:
    detect_outliers(train,i)

In [651]:
Y = train["Survived"]
X = train.drop(["Survived"],axis=1)
Xr_te = test.drop(["Survived"],axis=1)
print(data["Age"])


0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
413    28.0
414    39.0
415    38.5
416    28.0
417    28.0
Name: Age, Length: 1309, dtype: float64


In [652]:
for i in Xr_te.columns:
    print(X[i].isna().sum() , i)

0 Pclass
0 Sex
0 Age
0 SibSp
0 Parch
0 Fare


In [653]:
#X_tr,X_te,y_tr,y_te = train_test_split(X,Y,test_size=0.8,random_state=42,shuffle=True)

le = LogisticRegression()
le.fit(X,Y)



LogisticRegression()

In [654]:
filename = "/Users/hajiaga/Downloads/titanic/result.csv"
import os
for i in Xr_te.columns:
    print(Xr_te[i].isna().sum() , i)
gnd['Survived'] = le.predict(Xr_te).astype(int)
if os.path.exists(filename):
    os.remove(filename)
gnd.to_csv(filename, index=False) 
gnd.head()

0 Pclass
0 Sex
0 Age
0 SibSp
0 Parch
0 Fare


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
